In [1]:
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()
RddData = sc.textFile("FileStore/tables/Data.csv")
RddData.cache()
RddData.collect()

In [2]:
newRdd = RddData.filter(lambda x : "creditcd" not in x)
newRdd.collect()

In [3]:
from pyspark.ml.linalg import Vectors
def transformToVector (inp):
  list1 = inp.split(",")
  value = Vectors.dense([float(list1[11]), float(list1[0]), float(list1[1]) , float(list1[2]) , float(list1[3]) , float(list1[4]) , float(list1[5]) , float(list1[6]),\
                        float(list1[7]) , float(list1[8]) , float(list1[9]), float(list1[10])])
  return value
 

In [4]:
callVect = newRdd.map(transformToVector)
callVect.collect()

In [5]:
def tronsformToLabelPoint(inStr):
  label = (float(inStr[0]), Vectors.dense([inStr[1],inStr[2],inStr[3],inStr[4],inStr[5],inStr[6], inStr[7],inStr[8],inStr[9], inStr[10], inStr[11]]))
  return label
  

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
callLp = callVect.map(tronsformToLabelPoint)
callDF = sqlContext.createDataFrame(callLp, ["label" , "features"])
callDF.select("label", "features").show(5)

In [7]:
(trainingData, testData) = callDF.randomSplit([0.9, 0.1])
trainingData.count()


In [8]:
testData.count()

In [9]:

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [10]:
dtClassifer = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)
dtModel = dtClassifer.fit(trainingData)

In [11]:
predictions1 = dtModel.transform(testData)
  
predictions1.filter(predictions1['prediction'] == 0) \
    .select("label","features","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 23, truncate = 30)

In [12]:
prediction1.groupBy("label","prediction").count().show()

In [13]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("Test: Area Under ROC: " + str(evaluator.evaluate(predictions1, {evaluator.metricName: "areaUnderROC"})))
